## Module for Poles

In this module, we compile all functions needed to generate line shapes from different pole configurations. There is no need to run this notebook. The actual data generation shall be performed separately per pole configuration in their respective notebooks. Check the ```gendataset01_Pole``` notebook, and others.

In [ ]:
import math
import numpy as np
import cmath as cm
import random
import pickle
import os

### Define parameters

We define the global parameters to be used. This includes the hadron masses and the center-of-mass energy.

In [ ]:
# constants
hbarc = 197.3

# Heavy quark sector
# Units in MeV
JPsi = 3096.90
Proton = 938.27208816
Dplus = 1869.66
Dbar0 = 1864.83
Sigmaplus = 1189.37
Sigmaplus_C = 2452.9

# Define thresholds
T1 = (JPsi + Proton)/hbarc
T2 = (Sigmaplus_C + Dbar0)/hbarc
T4 = 4350/hbarc

We now define the energy region in which poles are to be located. We also specify the number of unique poles of different energies to generate. Specify the number of values for the real and imaginary parts of the poles in ```Nreal``` and ```Nimag```, respectively.

In [ ]:
Nreal = 100    # 100 for train  # 32 for test/validation sets
Nimag = 100    # 100 for train  # 10 for test/vdaliation sets

TotalN = Nreal*Nimag

NEpoints = 75  # number of energy points

# Define the counting region for main poles to be located
Erealbelow_in = np.random.uniform(low=T2*hbarc-50, high=T2*hbarc, size=int(Nreal/2))
Erealabove_in = np.random.uniform(low=T2*hbarc, high=T4*hbarc, size=int(Nreal/2))
Ereal = np.concatenate((Erealbelow_in, Erealabove_in))
Eimag = np.random.uniform(low=0.0, high=100.0, size=Nimag)

# Define the region beyond the counting region for backrgound poles to be located
Erealbelow_out = np.random.uniform(low=T1*hbarc-2000, high=T1*hbarc-100, size=int(Nreal/2))
Erealabove_out = np.random.uniform(low=T2*hbarc+500, high=T4*hbarc+5000, size=int(Nreal/2))
Erealfar = np.concatenate((Erealbelow_out, Erealabove_out))
Eimagfar = np.random.uniform(low=700.0, high=2000.0, size=Nimag)

We create a function ```gen_Eaxis()``` that can randomly generate energy points representative of each energy bin from the experimental data.

In [ ]:
# Input experimental data
LHCb_data = np.loadtxt('LHCb_Data.csv', skiprows=1, delimiter=',')
invmass = LHCb_data[:, 0]
invmass_low = LHCb_data[:, 1]
invmass_high = LHCb_data[:, 2]
weighted_candidates = LHCb_data[:, 3]
upper_err = LHCb_data[:, 4]
lower_err = LHCb_data[:, 5]

# Randomly generate energy points
def gen_Eaxis():
    x_data = []
    
    for i in range(len(invmass)):
        x_val = random.uniform(invmass_low[i],invmass_high[i])
        x_data.append(x_val)
    
    return x_data

# Define energy region of interest
energy_low = 4200.0
energy_high = 4350.0

### Create pole class

We create the class ```pole``` which creates an object that contains the pole position and Riemann sheet location, and a method ```pole.smat``` that gives the pole's S-matrix contribution.

In [ ]:
class unif_pole:
    def __init__(self, RS, Ereal, Eimag):
        # RS is the two-channel Reimann sheet location of the pole
        # Use [1,-1] for bt, [-1,-1] for bb and [-1,1] for tb
        # Ereal is the real part of energy pole
        # Eimag is the imag part of energy pole
        
        self.Ereal = Ereal
        self.Eimag = Eimag
        self.RS  = RS
        Epole = Ereal - (1j)*Eimag
        self.pos = Epole
        
        # Compute uniformized momentum pole
        # for channel 1 and channel 2
        k1pole = cm.sqrt((Epole/hbarc)**2-T1**2)
        k2pole = cm.sqrt((Epole/hbarc)**2-T2**2)
        
        # Riemann sheet assignment
        beta1  = RS[0]*abs(k1pole.imag)
        beta2  = RS[1]*abs(k2pole.imag)
        
        # Get the real part, we need to be consistent with signs
        alpha1 = -np.sign(beta1)*np.abs(k1pole.real)
        alpha2 = -np.sign(beta2)*np.abs(k2pole.real)

        # Just for counterchecking
        # signs of beta1 and beta2 should agree 
        # with RS[0] and RS[1], respectively
        self.alpha1 = alpha1
        self.alpha2 = alpha2
        self.beta1 = beta1
        self.beta2 = beta2
        
        # Construct pole channel momenta
        polep1 = (1j)*beta1 + alpha1
        polep2 = (1j)*beta2 + alpha2
        
        Delta = cm.sqrt(T2**2 - T1**2)
        self.Delta = Delta
        
        # Uniformization of the assigned pole
        omega_pole = (polep1 + polep2)/Delta
        recip_omg_pol = 1/omega_pole
        self.omega_pole = omega_pole
        self.recip_omg_pol = recip_omg_pol
        
        # Get pole regulator
        # for [bt] sheet
        if RS==[-1,1]: 
            omega_reg = np.abs(recip_omg_pol)*cm.exp(-0.5*np.pi*(1j))
        # for [bb] sheet
        elif RS==[-1,-1]:
            omega_reg = np.abs(recip_omg_pol)*cm.exp(-0.5*np.pi*(1j))
        # for [tb] sheet
        elif RS==[1,-1]:
            omega_reg = np.abs(recip_omg_pol)*cm.exp(-0.5*np.pi*(1j))
            
        self.omega_reg = omega_reg
        self.recip_omg_reg = 1/omega_reg
        
        p1_reg = omega_reg + 1/omega_reg
        p2_reg = omega_reg - 1/omega_reg
        
        # Riemann sheet identifier for pole regulator
        def RSlabel(pimag1, pimag2):
            if pimag1>0 and pimag2>0:
                RS = 'tt' #sheet 1
            elif pimag1<0 and pimag2>0:
                RS = 'bt' #sheet 2
            elif pimag1<0 and pimag2<0:
                RS = 'bb' #sheet 3
            elif pimag1>0 and pimag2<0:
                RS = 'tb' #sheet 4
            return RS
        # If you want to check the Riemann sheet of pole regulator
        self.regulator = ['{:.2f}'.format(np.sqrt(p1_reg**2.0+T1**2.0)*hbarc), RSlabel(p1_reg.imag, p2_reg.imag)]
        self.assignedpole = ['{:.2f}'.format(np.sqrt(polep1**2.0+T1**2.0)*hbarc), RSlabel(polep1.imag, polep2.imag)]
    
    # The indent on this part is very important
    # Calculate the S-matrix contribution of the uniformized pole
    def smat11(self, Ecm):
        # Get channel momenta of Ecm
        p1 = np.sqrt((Ecm/hbarc)**2.0-T1**2.0)
        p2 = np.zeros([NEpoints,],dtype = 'complex_')
        for pndx in range(len(Ecm)):
            p2_pts = cm.sqrt((Ecm[pndx]/hbarc)**2.0-T2**2.0)
            p2[pndx] = p2_pts

        # Get uniformized parameter
        omega = (p1 + p2)/self.Delta
        
        # Numerator of S-matrix
        Numpol = (omega-np.conj(self.recip_omg_pol))*(omega+self.recip_omg_pol)
        Numreg = (omega-np.conj(self.recip_omg_reg))*(omega+self.recip_omg_reg)
        Num = Numpol*Numreg
        
        # Denominator of S-matrix
        Denpol = (omega-self.omega_pole)*(omega+np.conj(self.omega_pole))
        Denreg = (omega-self.omega_reg)*(omega+np.conj(self.omega_reg))
        Den = Denpol*Denreg
            
        return np.abs((self.omega_pole*self.omega_reg))**2.0*Num/Den

Create ```skip_duplicate()``` function to avoid possible duplication of poles in the same Riemann sheet.

In [ ]:
def skip_duplicate(real1, imag1, Nreal, Nimag):
    # Create lists of available real and imag values
    real_list = [entry for entry in range(1, Nreal) if entry != real1]
    imag_list = [entry for entry in range(1, Nimag) if entry != imag1]

    # Randomly choose real values without duplication
    real_choices = np.random.choice(real_list, 5, replace=False)
    # Randomly choose imag values without duplication
    imag_choices = np.random.choice(imag_list, 5, replace=False)

    # Combine real and imag values into two lists
    real_values = [real1] + list(real_choices)
    imag_values = [imag1] + list(imag_choices)

    indx = [real_values, imag_values]
    return indx

###  Add phase space and background

We create a function ```dalitz()``` to generate the Dalitz plot in which the phase space shall be obtained.

In [ ]:
def dalitz(x_data):
    # Define hadron masses
    # Units in MeV
    ma = 5619.60        # (\Lambda_b^0)
    mb = 493.677        # (K^-)
    mc1 = JPsi
    mc2 = Proton
    
    # Convert mass to GeV
    ma = ma/1000
    mc1 = mc1/1000
    mb = mb/1000
    mc2 = mc2/1000

    N = len(x_data)

    # Dalitz plot axes
    exp_xmin = 2
    exp_xmax = 6.5
    
    m23sq = np.linspace(exp_xmin, exp_xmax, N)
    m13sq = (x_data/1000)**2

    # Construct Dalitz plot
    E_1 = (np.full(N, ma)**2 + np.full(N, mc1)**2 - m23sq)/(2*np.full(N, ma))
    E_2 = (np.full(N, ma)**2 + np.full(N, mb)**2 - m13sq)/(2*np.full(N, ma))
    X, Y = np.meshgrid(E_1, E_2)

    argument = 4*(X**2 - mc1**2)*(Y**2 - mb**2) - (ma**2 + mc1**2 + mb**2 - mc2**2 - 2*ma*(X + Y) + 2*X*Y)**2
    E_3 = ma - X - Y
    
    plot = argument*E_3 >= 0
    
    return plot

We also create a function ```polynom()``` to randomly generate an nth degree polynomial to be added as background.

In [ ]:
def polynom(x):
    # this is for a seventh degree polynomial
    coeff = np.random.uniform(0, 5, 7)
    total = np.sum([coeff[i]*(x**i) for i in range(len(coeff))], axis=0)
    norm = np.linalg.norm([total])
    poly_bg = total/norm
    
    return poly_bg